In [1]:
#Importing necessary libraries
import pandas as pd
import numpy as np

In [2]:
#Load in the data set
fifaDB = pd.read_csv("data.csv")

In [3]:
#Preview Data base
fifaDB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18207 entries, 0 to 18206
Data columns (total 89 columns):
Unnamed: 0                  18207 non-null int64
ID                          18207 non-null int64
Name                        18207 non-null object
Age                         18207 non-null int64
Photo                       18207 non-null object
Nationality                 18207 non-null object
Flag                        18207 non-null object
Overall                     18207 non-null int64
Potential                   18207 non-null int64
Club                        17966 non-null object
Club Logo                   18207 non-null object
Value                       18207 non-null object
Wage                        18207 non-null object
Special                     18207 non-null int64
Preferred Foot              18159 non-null object
International Reputation    18159 non-null float64
Weak Foot                   18159 non-null float64
Skill Moves                 18159 non-null fl

In [4]:
fifaDB.columns

Index(['Unnamed: 0', 'ID', 'Name', 'Age', 'Photo', 'Nationality', 'Flag',
       'Overall', 'Potential', 'Club', 'Club Logo', 'Value', 'Wage', 'Special',
       'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position',
       'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until',
       'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW',
       'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM',
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing',
       'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
       'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingT

In [5]:
#Drop unnecessary columns
Fifa = fifaDB.drop(['LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW',
       'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM',
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Unnamed: 0', 'Photo', 'Flag', 'Club Logo', 'Joined', 'Loaned From', 'Contract Valid Until', 'Special', 'International Reputation', 'Real Face', "Nationality", "Age", "ID", "Release Clause", "Club"], axis=1)

In [6]:
Fifa = Fifa[['Name', 'Overall', 'Potential', 'Value', 'Wage', 'Preferred Foot',
       'Weak Foot', 'Skill Moves', 'Work Rate', 'Body Type',
       'Jersey Number', 'Height', 'Weight', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes', 'Position']]

In [7]:
#Look for null values
Fifa.isna().sum() 

Name                0
Overall             0
Potential           0
Value               0
Wage                0
Preferred Foot     48
Weak Foot          48
Skill Moves        48
Work Rate          48
Body Type          48
Jersey Number      60
Height             48
Weight             48
Crossing           48
Finishing          48
HeadingAccuracy    48
ShortPassing       48
Volleys            48
Dribbling          48
Curve              48
FKAccuracy         48
LongPassing        48
BallControl        48
Acceleration       48
SprintSpeed        48
Agility            48
Reactions          48
Balance            48
ShotPower          48
Jumping            48
Stamina            48
Strength           48
LongShots          48
Aggression         48
Interceptions      48
Positioning        48
Vision             48
Penalties          48
Composure          48
Marking            48
StandingTackle     48
SlidingTackle      48
GKDiving           48
GKHandling         48
GKKicking          48
GKPosition

In [8]:
#Drop null values
Fifa = Fifa.dropna()

In [9]:
list_lengths = np.array(Fifa['Height'])
empty_list = []
for x in list_lengths:
    splitted_values = x.split("'")
    feet = (float(x[0])) + (float(x[2])/10)
    centimeters = feet * 30.48
    empty_list.append(round(centimeters))

Fifa['Height'] = empty_list

In [10]:
Fifa['Height']

0        174
1        189
2        180
3        195
4        155
        ... 
18202    180
18203    192
18204    177
18205    155
18206    155
Name: Height, Length: 18147, dtype: int64

In [11]:
Fifa["Body Type"].unique()

array(['Messi', 'C. Ronaldo', 'Neymar', 'Lean', 'Normal', 'Courtois',
       'Stocky', 'PLAYER_BODY_TYPE_25', 'Shaqiri', 'Akinfenwa'],
      dtype=object)

In [12]:
#onehot encoding
obj_columns = list(Fifa.select_dtypes(include=["object"]).columns)
obj_columns = obj_columns[3:6]
dfencoded = pd.get_dummies(data=Fifa, columns=obj_columns)

In [13]:
def savetocsv(dataframe, name):
    dataframe.to_csv(name, index = None, header=True)

In [14]:
savetocsv(dfencoded, "NewFifa.csv")